In [1]:
from jupyter_dash import JupyterDash 
from dash import dcc
from dash import html
from dash.dependencies import Input, Output

app = JupyterDash('myFirstReactive')

app.layout = html.Div([
    html.P(children="Input: "),
    dcc.Input(id='inum', value='', type='number'),
    html.P(id='outext')
])

@app.callback(
    Output(component_id='outext', component_property='children'),
    Input(component_id='inum', component_property='value')
)
def div_outext(num):
    return f'times two: {num*2}'

#app.run_server(mode='inline')

In [2]:
from dash.dependencies import Input, Output

app = JupyterDash()

app.layout = html.Div([
    html.Div(["Enter first and last name: ",
    dcc.Input(id='fname_i', value='otto', type='text'),
    dcc.Input(id='lname_i', value='renner', type='text')]),
    html.Br(),
    html.Div([html.P(id='fname_o', style={'float': 'left'}),
    html.P(id='lname_o', style={'float': 'left','marginLeft': 4})] )
])
    
@app.callback(
    Output(component_id='fname_o', component_property='children'),
    Output(component_id='lname_o', component_property='children'),
    Input(component_id='fname_i', component_property='value'),
    Input(component_id='lname_i', component_property='value')
)
def div_outext(fname, lname):
    if fname:
        return fname[::-1], lname[::-1]
    
#app.run_server(mode='inline')

In [3]:
import pandas as pd
import datetime as dt
import plotly.express as px
df = pd.read_csv('data/dji_sector_prices.csv')
df.date = pd.to_datetime(df.date)
df = df[df.date.dt.year == 2021]
df['month'] = [el.month for el in df.date]

In [4]:
month_marks = {i+1:str(i+1) for i in range(df.month.max())}

In [5]:
app = JupyterDash()

app.layout = html.Div([
    dcc.Slider(
        id='comp_slider',
        min= min(df.month),
        max= max(df.month),
        marks=month_marks,
        value=5
        ),
    dcc.Dropdown(id='company_dd',
                options = [{'label': i, 'value': i} for i in df.symbol.unique()],
                value = df.symbol.unique()[0]),
    dcc.Graph(id='outgraph')
])

@app.callback(
    Output('outgraph', 'figure'),
    Input('comp_slider', 'value'),
    Input('company_dd', 'value')
)
def make_outgraph(month, sym):
    dftemp = df[df.symbol == sym]
    dftemp = dftemp[dftemp.month == month]
    fig = px.line(dftemp, x='date', y='open')
    return fig

#app.run_server(mode='inline)

In [6]:
from dash.dependencies import State

app = JupyterDash()

white_button_style = {'background-color': 'grey',
                      'color': 'white',
                      'height': '50px',
                      'width': '100px',
                      'margin-top': '5px',
                      'margin-left': '5px'}

app.layout = html.Div([
    dcc.Slider(
        id='comp_slider',
        min= min(df.month),
        max= max(df.month),
        marks=month_marks,
        value=5
        ),
    dcc.Dropdown(id='company_dd',
                options = [{'label': i, 'value': i} for i in df.symbol.unique()],
                value = df.symbol.unique()[0], style=dict(width='400px')),
    html.Button('Update', id='update', n_clicks=0, style=white_button_style),
    dcc.Graph(id='outgraph')
])

@app.callback(
    Output('outgraph', 'figure'),
    Input('update', 'n_clicks'),
    State('comp_slider', 'value'),
    State('company_dd', 'value')
)
def make_outgraph(n_clicks, month, sym):
    dftemp = df[df.symbol == sym]
    dftemp = dftemp[dftemp.month == month]
    fig = px.line(dftemp, x='date', y='open')
    return fig

#app.run_server(mode='inline')

In [7]:
app = JupyterDash()

app.layout = html.Div([
    dcc.Dropdown(id='sector_dd',
                options = [{'label': i, 'value': i} for i in df.sector.unique()],
                value = df.sector.unique()[0]),
    dcc.Dropdown(id='company_dd',
                value = df.symbol.unique()[0])
])

@app.callback(
    Output('company_dd', 'options'),
    Input('sector_dd', 'value')
)
def set_options_company_dd(sector):
    return [{'label': i, 'value': i} for i in df[df.sector==sector].symbol.unique()]

@app.callback(
    Output('company_dd', 'value'),
    Input('company_dd', 'options')
)
def set_default_company_dd(ops):
    return ops[0]['value']
    
#app.run_server(mode='inline')

In [8]:
from dash import dash_table
import plotly.graph_objects as go
import dash

In [9]:
app = JupyterDash()


def build_figure(df):
    fig = go.Figure(go.Scatter(x=df.open, y=df.change,
                       marker={'color': 'gray'},
                       mode='markers',
                       selected={'marker': {'color': 'Crimson'}},
                       unselected={'marker': {'opacity': 0.4}}),
        )
    fig.update_layout(dragmode='lasso')
    return fig

app.layout = html.Div([
    dcc.Graph(id='graph', figure=build_figure(df[df.symbol == 'PG'])),
    dash_table.DataTable(id='table',
                         columns = [{'name': col, 'id': col} for col in df.columns])

])

@app.callback(
    Output('table', 'data'),
    [Input("graph", "selectedData"),
     Input("graph", "clickData")])
def update_table(selectedData, clickData):
    selection = None
    data = None
    trigger = dash.callback_context.triggered[0]["prop_id"]
    if trigger == 'graph.clickData':
        selection = [point["pointIndex"] for point in clickData["points"]]
        data=df.iloc[selection].to_dict(orient='records')

    if trigger == 'graph.selectedData':
        selection = [point["pointIndex"] for point in selectedData["points"]]
        data=df.iloc[selection].to_dict(orient='records')
        
    print('\n', dash.callback_context.triggered)
    return data

#app.run_server(mode='inline')

In [10]:
def text_to_wrap():
    print('my text')
    
text_to_wrap()

my text


In [11]:
def emphasize_decorator(func):
    def wrapper():
        print('##################')
        func()
        print('!!!!!!!!!!!!!!!!!!')
    return wrapper

In [12]:
text_to_wrap = emphasize_decorator(text_to_wrap)

In [13]:
text_to_wrap()

##################
my text
!!!!!!!!!!!!!!!!!!


In [14]:
# emphasize_decorator is being treated as already defined

@emphasize_decorator
def print_greeting():
    print('Hello')

In [15]:
print_greeting()

##################
Hello
!!!!!!!!!!!!!!!!!!


In [16]:
def emphasize_decorator(func):
    def wrapper(*args, **kwargs):
        print('##################')
        func(*args, **kwargs)
        print('!!!!!!!!!!!!!!!!!!')
    return wrapper

In [17]:
@emphasize_decorator
def text_to_wrap(w1, w2):
    print(f"{w1}\n{w2}")

In [18]:
text_to_wrap('line1', 'line2')

##################
line1
line2
!!!!!!!!!!!!!!!!!!
